In [1]:
from langgraph.prebuilt import ToolNode


def weather_forecast(location: str) -> str:
    """This function provides the weather forecast.

    Args:
        location: (str) weather forecast location.
    """
    return f"The weather in {location} is 36 degrees sunny."


def get_the_time(location: str) -> str:
    """This tool provides the time for a given location.

    Args:
    location: (str) the location to get the time.
    """
    return f"Response: The time in {location} is 6pm."


tools = [weather_forecast, get_the_time]

tool_node = ToolNode(tools)

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1", temperature=0).bind_tools(tools)

In [3]:
from langgraph.graph import MessageGraph
from langgraph.prebuilt import tools_condition


class Steps:
    CHAT = "chatbot"
    TOOLS = "tools"
    FINAL = "final"


builder = MessageGraph()

builder.set_entry_point(Steps.CHAT)
builder.add_node(Steps.TOOLS, tool_node)
builder.add_node(Steps.CHAT, llm)
builder.add_edge(Steps.TOOLS, Steps.CHAT)

builder.add_conditional_edges(Steps.CHAT, tools_condition)

graph = builder.compile()

In [4]:
for s in graph.stream(
    (
        "human",
        "what is 2 + 2? Also what is the weather in Paris? Also what is 3 + 3 + 3 and then what is the time in london?",
    ),
    stream_mode="values",
):
    s[-1].pretty_print()

================================ Human Message =================================

what is 2 + 2? Also what is the weather in Paris? Also what is 3 + 3 + 3 and then what is the time in london?
================================== Ai Message ==================================
Tool Calls:
  weather_forecast (d36631b5-e305-4d33-9459-885fac372cce)
 Call ID: d36631b5-e305-4d33-9459-885fac372cce
  Args:
    location: Paris
  get_the_time (bc021855-b2c8-4453-8cd0-20d4bad17c6a)
 Call ID: bc021855-b2c8-4453-8cd0-20d4bad17c6a
  Args:
    location: London
================================= Tool Message =================================
Name: get_the_time

Response: The time in London is 6pm.
================================== Ai Message ==================================
Tool Calls:
  weather_forecast (7645211a-aa5a-4c62-af1e-09b7174ae19b)
 Call ID: 7645211a-aa5a-4c62-af1e-09b7174ae19b
  Args:
    location: Paris
  get_the_time (24fa9209-8295-4500-b25e-1abeeb9aac68)
 Call ID: 24fa9209-8295-4500-b25e-